In [1]:
from PIL import Image
from pytesseract import image_to_string
import pytesseract
import numpy as np
import pandas as pd
import PIL
from PIL import Image, ImageFilter, ImageEnhance
import cv2
import colorsys

In [29]:
 pytesseract.pytesseract.tesseract_cmd = r"C:\Users\ankur\AppData\Local\Tesseract-OCR\tesseract.exe"

In [30]:
import numpy as np
#initial coordinates of name, guardian's relation and name,house number,age,gender.
#Pls edit and change by hit and try like we did in first page extraction
def image_text(x,
               crop_rectangle_hn = (400, 470, 825, 540),crop_rectangle_age = (310, 540, 455, 585),
               crop_rectangle_gen = (560, 540, 585, 585),crop_rectangle_g_relation = (220, 400, 247, 440)):
    im = Image.open(x)
    im = im.resize((3000, 3882), PIL.Image.LANCZOS)
    im = cv2.adaptiveThreshold(np.array(im,dtype=np.uint8), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 9)
    im = Image.fromarray(im)
    countt = 1   
    for i in range(0,10):
# Split file name like we did in first page extraction. It should include full file name.
# For example 'AC0020001-03' and not just 'AC0020001'
        for j in range(0,3):
#             s = x.split('/')
#             s = s[1].split("_")[0]
            file_name.append(x+str(countt))
            
            cropped_im_g_relation = im.crop(crop_rectangle_g_relation)
            c_gr = image_to_string(cropped_im_g_relation,config='-c tessedit_char_whitelist=FH --psm 6')
            grel.append(c_gr)

            cropped_im_hn = im.crop(crop_rectangle_hn)
            c_hn = image_to_string(cropped_im_hn,config='--psm 6')
            hno.append(c_hn)

            cropped_im_age = im.crop(crop_rectangle_age)
            c_age = image_to_string(cropped_im_age,config='-c tessedit_char_whitelist=0123456789 --psm 6')
            age.append(c_age)

            cropped_im_gen = im.crop(crop_rectangle_gen)
            c_gen = image_to_string(cropped_im_gen,config='-c tessedit_char_whitelist=MF --psm 6')
            gen.append(c_gen)
            
            countt+=1

# Find distance between same category horizontally.
# For example find distance between starting coordinate of first box NAME and starting coordinate of secnd box NAME on right
# Make sure we find distance from starting point. Distance should be same for 2nd and 3rd box horizontally as well as for every 4 other value, such as age, sex etc
            crop_rectangle_hn = tuple(np.add(crop_rectangle_hn, (863.5, 0,863.5,0)))
            crop_rectangle_age = tuple(np.add(crop_rectangle_age, (863.5, 0,863.5,0)))
            crop_rectangle_gen = tuple(np.add(crop_rectangle_gen, (863.5, 0,863.5,0)))
            crop_rectangle_g_relation = tuple(np.add(crop_rectangle_g_relation, (863.5, 0,863.5,0)))
# Find ditance between third and first box. Which should be 3 times above found distance

        crop_rectangle_hn = tuple(np.subtract(crop_rectangle_hn, (2590.5, 0,2590.5,0)))
        crop_rectangle_age = tuple(np.subtract(crop_rectangle_age, (2590.5, 0,2590.5,0)))
        crop_rectangle_gen = tuple(np.subtract(crop_rectangle_gen, (2590.5, 0,2590.5,0)))
        crop_rectangle_g_relation = tuple(np.subtract(crop_rectangle_g_relation, (2590.5, 0,2590.5,0)))
# Find distance between same category vertically.
# For example find distance between starting coordinate of first box NAME and starting coordinate of secnd box NAME down
# Distance should be same for all 10 boxes vertically as well as for every 4 other value, such as age, sex etc

        crop_rectangle_hn = tuple(np.add(crop_rectangle_hn, (0, 337,0,337)))
        crop_rectangle_age = tuple(np.add(crop_rectangle_age, (0, 337,0,337)))
        crop_rectangle_gen = tuple(np.add(crop_rectangle_gen, (0, 337,0,337)))
        crop_rectangle_g_relation = tuple(np.add(crop_rectangle_g_relation, (0, 337,0,337)))

    return hno,age,gen,file_name,grel



** Removing page 1 and 2 **

In [31]:
import glob
#Getting all page 1
page_1 = glob.glob("C:/Users/ankur/New folder/*01.jpg")
#Getting all page 2
page_2 = glob.glob('C:/Users/ankur/New folder/*02.jpg')
#Getting all pages
page_all = glob.glob('C:/Users/ankur/New folder/*.jpg')

In [32]:
#Removing page 1 from all
temp = [i for i in page_all if i not in page_1]
#Removing page 2 from all
page_fin = [i for i in temp if i not in page_2]

In [33]:
# with open('drive/My Drive/app/Misc/aap/get_pdfs/images/one.txt', 'w') as f:
#     for item in page_fin:
#         f.write("%s\n" % item)

# from numpy import loadtxt
# one = loadtxt("drive/My Drive/app/Misc/aap/get_pdfs/images/one.txt", delimiter="\n", unpack=False,dtype = 'str')

In [34]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

** Splitting list to parts **

In [35]:
#Splitting files into 500 parts to increase frequency of saving. Increase the value to save more frequently
#Do not change value after running code once. If want to change value after running. Delete all saved files or do maths
page_fin_split =  list(split(page_fin, 500))

**WITH TESSERACT 3.05**

In [36]:
# Count value is the starting value for code to work. If the last file saved before stopping code was '10', count value for running code again should be '11'
count = 467
count_sem =467
for i in page_fin_split[count-1:]:
    name = []
    father = []
    hno = []
    age = []
    gen = []
    grel = []
#     code = []
    file_name = []
    for j in i:
        h,a,g,fn,gr = image_text(j)
        print("page_"+str(count_sem))
        count_sem+=1
    
    df = pd.DataFrame(np.column_stack([gr, h,a,g,fn]), 
                               columns=['g_relation', 'hno','age','gen',"file_name"])
    df.to_csv('image'+str(count)+".csv",index = False)
    print(count)
    count+=1

KeyboardInterrupt: 

**WITH TESSERACT 4.0**

In [41]:
import numpy as np
#initial coordinates of name, guardian's relation and name,house number,age,gender.
#Pls edit and change by hit and try like we did in first page extraction
def image_text_name(x, crop_rectangle_hn = (400, 470, 825, 540),crop_rectangle_nm = (340, 320, 820, 410),crop_rectangle_fn = (410, 400, 825, 480)):
    im = Image.open(x)
    im = im.resize((3000, 3882), PIL.Image.LANCZOS)
    im = cv2.adaptiveThreshold(np.array(im,dtype=np.uint8), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 9)
    im = Image.fromarray(im)
    countt = 1
    for i in range(0,10):
# Split file name like we did in first page extraction. It should include full file name.
# For example 'AC0020001-03' and not just 'AC0020001'
        for j in range(0,3):
#             s = x.split('/')
#             s = s[1].split("_")[0]
            file_name.append(x+str(countt))
    
            cropped_im_nm = im.crop(crop_rectangle_hn)
            c_hn = image_to_string(cropped_im_nm,config='-c tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyzQWERTYUIOPASDFGHJKLZXCVBNM --psm 6')
            hno.append(c_hn)

            cropped_im_nm = im.crop(crop_rectangle_nm)
            c_nm = image_to_string(cropped_im_nm,config='-c tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyzQWERTYUIOPASDFGHJKLZXCVBNM --psm 6')
            name.append(c_nm)
            
            cropped_im_fn = im.crop(crop_rectangle_fn)
            c_fn = image_to_string(cropped_im_fn,config='-c tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyzQWERTYUIOPASDFGHJKLZXCVBNM --psm 6')
            father.append(c_fn)
            
            countt+=1


# Find distance between same category horizontally.
# For example find distance between starting coordinate of first box NAME and starting coordinate of secnd box NAME on right
# Make sure we find distance from starting point. Distance should be same for 2nd and 3rd box horizontally as well as for every 4 other value, such as age, sex etc
            crop_rectangle_nm = tuple(np.add(crop_rectangle_nm, (863.5, 0,863.5,0)))
            crop_rectangle_fn =  tuple(np.add(crop_rectangle_fn, (863.5, 0,863.5,0)))
            crop_rectangle_hn = tuple(np.add(crop_rectangle_hn, (863.5, 0,863.5,0)))

# Find ditance between third and first box. Which should be 3 times above found distance
 
        crop_rectangle_hn = tuple(np.subtract(crop_rectangle_hn, (2590.5, 0,2590.5,0)))
        crop_rectangle_nm = tuple(np.subtract(crop_rectangle_nm, (2590.5, 0,2590.5,0)))
        crop_rectangle_fn =  tuple(np.subtract(crop_rectangle_fn, (2590.5, 0,2590.5,0)))

# Find distance between same category vertically.
# For example find distance between starting coordinate of first box NAME and starting coordinate of secnd box NAME down
# Distance should be same for all 10 boxes vertically as well as for every 4 other value, such as age, sex etc
        crop_rectangle_nm = tuple(np.add(crop_rectangle_nm, (0, 337,0,337)))
        crop_rectangle_fn =  tuple(np.add(crop_rectangle_fn, (0, 337,0,337)))
        crop_rectangle_hn = tuple(np.add(crop_rectangle_hn, (0, 337,0,337)))
    return name,father,file_name,hno



In [ ]:
# Count value is the starting value for code to work. If the last file saved before stopping code was '10', count value for running code again should be '11'
count = 1
count_sem = 1
for i in page_fin_split[count-1:]:
    name = []
    father = []
    hno = []
    age = []
    gen = []
    grel = []
#     code = []
    file_name = []
    for j in i:
        n,f,fn,h = image_text_name(j)

        print("page_"+str(count_sem))
        count_sem+=1
    
    df = pd.DataFrame(np.column_stack([n, f,fn,h]), 
                               columns=['name', 'guardian','file_name','hno'])
    #Put it in different folder
    df.to_csv('image1'+str(count)+".csv",index = False)
    print(count)
    count+=1

page_1
page_2
1
page_3
page_4
2
page_5
page_6
3
page_7
